In [1]:
import ollama
import json # To work with JSON output from LLM
import pandas as pd
import os

# --- Re-Load necessary data (if kernel restarted or running this cell independently) ---
data_folder = ''
graph_file_path = os.path.join(data_folder, "road_network_Delhi_India.graphml")
# Ensure osmnx is loaded with the correct version (1.2.0)
import osmnx as ox
import networkx as nx

try:
    # We explicitly load the graph here
    G = ox.load_graphml(filepath=graph_file_path)
    print("Road network graph loaded for LLM integration.")
except Exception as e:
    print(f"Error loading graph for LLM integration: {e}")
    print("Please ensure the graph was saved correctly in Step 4 and path is accurate.")
    exit()

aligned_ev_file_path = os.path.join(data_folder, 'aligned_ev_stations.csv')
try:
    df_stations = pd.read_csv(aligned_ev_file_path)
    # Ensure 'nearest_node' is int type
    df_stations['nearest_node'] = df_stations['nearest_node'].astype(int)
    print("Aligned EV stations data loaded for LLM integration.")
except Exception as e:
    print(f"Error loading aligned EV stations for LLM integration: {e}")
    print("Please ensure aligned_ev_stations.csv was saved correctly in Step 5 and path is accurate.")
    exit()


# --- Define LLM Interaction Function ---
def get_trip_parameters_from_llm(user_query):
    """
    Uses the local Ollama LLM to extract structured trip parameters from a user query.

    Args:
        user_query (str): The natural language query from the user (e.g., "Plan a trip from Delhi to Noida").

    Returns:
        dict: A dictionary containing 'origin', 'destination', 'ev_model' (or None if not found),
              or an error message.
    """
    # Define the prompt for the LLM. This is crucial for guiding its output.
    # We instruct it to output JSON for easy parsing.
    prompt = f"""
    You are an AI assistant that helps extract trip information from user queries for an EV journey planner.
    Your task is to identify the ORIGIN, DESTINATION, and optionally the EV_MODEL.
    If an EV_MODEL is not mentioned, use "Tata Nexon EV" as the default.
    If you cannot identify a clear origin or destination, state "incomplete_query".

    Respond ONLY with a JSON object. Do NOT include any other text or markdown outside the JSON.

    Example 1:
    User: "Plan a trip from Connaught Place to Noida Sec 62 for my Tata Tiago EV."
    JSON: {{"origin": "Connaught Place, Delhi", "destination": "Noida Sec 62", "ev_model": "Tata Tiago EV"}}

    Example 2:
    User: "I want to travel from Mumbai to Pune."
    JSON: {{"origin": "Mumbai", "destination": "Pune", "ev_model": "Tata Nexon EV"}}

    Example 3:
    User: "Find charging stations near Connaught Place."
    JSON: {{"origin": "Connaught Place, Delhi", "destination": "Connaught Place, Delhi", "ev_model": "Tata Nexon EV", "intent": "find_stations"}}

    Example 4:
    User: "What's the best route?"
    JSON: {{"origin": null, "destination": null, "ev_model": null, "error": "incomplete_query"}}

    User Query: "{user_query}"
    JSON:
    """
    try:
        # Call the local Ollama model
        response = ollama.chat(model='phi3', messages=[
            {'role': 'system', 'content': prompt},
            {'role': 'user', 'content': user_query}
        ], options={'temperature': 0.1}) # Use lower temperature for more predictable output

        # Extract the content from the response
        llm_response_content = response['message']['content'].strip()
        print(f"LLM Raw Response: \n{llm_response_content}")

        # Try to parse the JSON response
        parsed_json = json.loads(llm_response_content)
        return parsed_json

    except json.JSONDecodeError as e:
        print(f"Error parsing LLM response as JSON: {e}")
        print(f"Problematic content: {llm_response_content}")
        return {"error": "LLM response not valid JSON. Please try again or refine prompt."}
    except Exception as e:
        print(f"An unexpected error occurred during LLM interaction: {e}")
        return {"error": f"LLM interaction failed: {e}"}

# --- Test the LLM Integration ---
print("\n--- Testing LLM-based Natural Language Understanding (NLU) ---")

# Test Query 1: Basic trip planning
query_1 = "I want to drive from Dilli Haat to Qutub Minar in my Tata Punch EV."
params_1 = get_trip_parameters_from_llm(query_1)
print(f"\nQuery 1 Parsed Parameters: {params_1}")

# Test Query 2: Missing EV model, default should be applied
query_2 = "What is the best route from Red Fort to India Gate?"
params_2 = get_trip_parameters_from_llm(query_2)
print(f"\nQuery 2 Parsed Parameters: {params_2}")

# Test Query 3: Incomplete query
query_3 = "Just tell me about routes."
params_3 = get_trip_parameters_from_llm(query_3)
print(f"\nQuery 3 Parsed Parameters: {params_3}")

# Test Query 4: Find stations intent
query_4 = "Find free charging stations near South Extension market."
params_4 = get_trip_parameters_from_llm(query_4)
print(f"\nQuery 4 Parsed Parameters: {params_4}")

print("\n--- LLM Integration (NLU) step complete. ---")

Road network graph loaded for LLM integration.
Aligned EV stations data loaded for LLM integration.

--- Testing LLM-based Natural Language Understanding (NLU) ---
An unexpected error occurred during LLM interaction: model requires more system memory (5.6 GiB) than is available (3.3 GiB) (status code: 500)

Query 1 Parsed Parameters: {'error': 'LLM interaction failed: model requires more system memory (5.6 GiB) than is available (3.3 GiB) (status code: 500)'}
An unexpected error occurred during LLM interaction: model requires more system memory (5.6 GiB) than is available (3.3 GiB) (status code: 500)

Query 2 Parsed Parameters: {'error': 'LLM interaction failed: model requires more system memory (5.6 GiB) than is available (3.3 GiB) (status code: 500)'}
An unexpected error occurred during LLM interaction: model requires more system memory (5.6 GiB) than is available (3.3 GiB) (status code: 500)

Query 3 Parsed Parameters: {'error': 'LLM interaction failed: model requires more system me